In [1]:
import pandas as pd
import ast
import numpy as np
import json

In [2]:
import os
print(os.getcwd())

c:\Users\Pc - Jujuy\Desktop\proyercto individual23\PI_ML_OPS


In [3]:
# Leer el archivo 
file_path = "movies_dataset.csv"
df_raw1 = pd.read_csv(file_path)


C:\Users\Pc - Jujuy\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3508: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
print(df_raw1['genres'].head())

0    [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1    [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2    [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                       [{'id': 35, 'name': 'Comedy'}]
Name: genres, dtype: object


In [5]:
df_raw1.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,Toy Story Collection,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [6]:
df_raw1.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,Toy Story Collection,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [7]:
columns_names = df_raw1.columns.values
columns_names

array(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'id', 'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'], dtype=object)

In [8]:
# define columns with data of dict type to process
cols = ['genres']

In [9]:
s = df_raw1[cols[0]].copy()
s.shape

(45466,)

In [10]:
s[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [11]:
# evaluate as a list
l = ast.literal_eval(s[0])
l

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [12]:
# copy one column to a pandas series
s = df_raw1[cols[0]].copy()
# fillna with [None]
s.fillna('[{}]', inplace=True)

l = []  # init an empty list

for i in s:
    if i == [{}]:
        # append [{}] to the list
        l += i
    else:
        # evaluate as a list
        l += ast.literal_eval(i)

In [13]:
len(l)

91106

In [14]:
l[:10]

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 10751, 'name': 'Family'},
 {'id': 10749, 'name': 'Romance'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 18, 'name': 'Drama'}]

In [15]:
def to_list_of_dict(series):
    """
    Evaluate a pandas series as a list of dict

    Input:
    "[{'one': 1, 'two': 2, 'three': 3}]"

    Output:
    [{'one': 1,
      'two': 2,
      'three' : 3}]
    """
    l = []  # init an empty list
    s = series.fillna('[{}]')  # map nan to [{}] for further eval

    # loop through the whole series
    for i in s:
        if i == [{}]:
            # append [{}] to the list
            l += i
        else:
            # evaluate as a list
            l += ast.literal_eval(i)

    return l

In [16]:
def column_conversion(col, df):
    """
    Merge a pandas series with data like list of dict back to the dataframe

    Input:
    "[{'one': 1, 'two': 2, 'three': 3}]"

    Output:
    A dataframe with the original column removed, each dict's key in a new column
    """
    l = to_list_of_dict(df[col])  # convert to list of dict
    df_right = pd.DataFrame.from_dict(l)  # convert to df
    df_merged = df.merge(df_right.add_prefix(col+'_'),  # add the original column name as prefix
                         left_index=True, right_index=True)  # merge df with df_right
    df_merged.drop(col, axis=1, inplace=True) # drop the original column

    
    return df_merged

In [17]:
# Test
column_conversion(cols[0], df_raw1)[:1]


,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,genres_id,genres_name
0,False,Toy Story Collection,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,16,Animation


In [19]:
# Guardar el DataFrame en un archivo CSV
df_raw1.to_csv('movies_dataset.csv', index=False)